In [41]:
import pandas as pd
import numpy as np

# Load the two datasets into pandas dataframes

In [42]:
## Loading the dataset
data = np.loadtxt("data/HT_Sensor_dataset.dat", skiprows=1)

dataset = pd.DataFrame(
    {'id': data[:, 0],
     'time': data[:, 1],
     'r1': data[:, 2],
     'r2': data[:, 3],
     'r3': data[:, 4],
     'r4': data[:, 5],
     'r5': data[:, 6],
     'r6': data[:, 7],
     'r7': data[:, 8],
     'r8': data[:, 9],
     'temp': data[:, 10],
     'humidity': data[:, 11]
    })
dataset['id'] = dataset['id'].astype(int)

In [43]:
## Importing dataset
metadata = np.loadtxt("data/HT_Sensor_metadata.dat", skiprows=1, dtype=str)

metadataset = pd.DataFrame(
    {'id': metadata[:, 0],
     'date': metadata[:, 1],
     'class': metadata[:, 2],
     'start_time_hours': metadata[:, 3],
     'duration_hours': metadata[:, 4]
    })
metadataset['id'] = metadataset['id'].astype(int)

#Apparently no data
metadataset = metadataset[metadataset['id'] != 95]

In [44]:
joined_df = pd.merge(dataset, metadataset, left_on='id', right_on='id', how='inner')
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928991 entries, 0 to 928990
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                928991 non-null  int64  
 1   time              928991 non-null  float64
 2   r1                928991 non-null  float64
 3   r2                928991 non-null  float64
 4   r3                928991 non-null  float64
 5   r4                928991 non-null  float64
 6   r5                928991 non-null  float64
 7   r6                928991 non-null  float64
 8   r7                928991 non-null  float64
 9   r8                928991 non-null  float64
 10  temp              928991 non-null  float64
 11  humidity          928991 non-null  float64
 12  date              928991 non-null  object 
 13  class             928991 non-null  object 
 14  start_time_hours  928991 non-null  object 
 15  duration_hours    928991 non-null  object 
dtypes: float64(11), int6

In [45]:
def change_class(row):
    if row['time'] < 0:
        return 'background'
    else:
        return row['class']

joined_df['class'] = joined_df.apply(change_class, axis=1)
joined_df.head()

In [46]:
X = []
y = []
## How many rows will be used for the prediction
snippet_size = 120
expected_rows_per_minute = 60
expected_snippet_time_gap = 1.05 * snippet_size / expected_rows_per_minute / 60
## How many rows will be skipped between each snippet
gap_size = 12
for start_index in range(0, len(joined_df) - snippet_size, gap_size):
    end_index = start_index + snippet_size
    ## Skip if the data spans across segments
    if joined_df.iloc[end_index,0] != joined_df.iloc[start_index,0]:
        continue
    ## If the time difference from beginning to end is larger than expected
    ## there must be some data missing or this chunk spans across tests
    if joined_df.iloc[end_index,1] - joined_df.iloc[start_index,1] > expected_snippet_time_gap:
        continue
    snippet = joined_df.iloc[start_index:end_index, 2:12].values
    X.append(snippet)
    y.append(joined_df.iloc[end_index,13])
    
X = np.array(X)
y = np.array(y)

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Remove rows from X_train and y_train where the second column is less than 0
## This is to remove the background data which was tilting the model to over prioritize background data
y_train = y_train[X_train[:,:,0] >= 0]
X_train = X_train[X_train[:,:,0] >= 0]

print(X_train.shape)
print(y_train)

(37909, 120, 10)
['wine' 'wine' 'banana' ... 'background' 'background' 'banana']


In [48]:
from sklearn.metrics import classification_report

from sktime.classification.deep_learning.lstmfcn import LSTMFCNClassifier
import pickle

model = LSTMFCNClassifier(n_epochs=20, verbose=True)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
pred = model.predict(X_test)
print(classification_report(pred, y_test))
pickle.dump(model, open("trained_model", "wb"))

/Users/dapomeranz/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 10, 120)]            0         []                            
                                                                                                  
 conv1d_11 (Conv1D)          (None, 10, 128)              123008    ['input_5[0][0]']             
                                                                                                  
 batch_normalization_9 (Bat  (None, 10, 128)              512       ['conv1d_11[0][0]']           
 chNormalization)                                                                                 
                                                                                                  
 activation_9 (Activation)   (None, 10, 128)              0         ['batch_normalization_9[

2023-10-19 15:46:00.017097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-19 15:46:00.836729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-19 15:46:06.938917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


297/297 [==============================] - 68s 178ms/step - loss: 0.6196 - accuracy: 0.7370
Epoch 2/20
297/297 [==============================] - 32s 107ms/step - loss: 0.4730 - accuracy: 0.8057
Epoch 3/20
297/297 [==============================] - 35s 119ms/step - loss: 0.3983 - accuracy: 0.8432
Epoch 4/20
297/297 [==============================] - 38s 127ms/step - loss: 0.3235 - accuracy: 0.8779
Epoch 5/20
297/297 [==============================] - 39s 133ms/step - loss: 0.2696 - accuracy: 0.9022
Epoch 6/20
297/297 [==============================] - 33s 110ms/step - loss: 0.2185 - accuracy: 0.9211
Epoch 7/20
297/297 [==============================] - 35s 117ms/step - loss: 0.1976 - accuracy: 0.9310
Epoch 8/20
297/297 [==============================] - 33s 112ms/step - loss: 0.1707 - accuracy: 0.9408
Epoch 9/20
297/297 [==============================] - 34s 116ms/step - loss: 0.1400 - accuracy: 0.9515
Epoch 10/20
297/297 [==============================] - 35s 117ms/step - loss: 0.1252

2023-10-19 15:57:56.349770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-19 15:57:56.687682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


75/75 [==============================] - 7s 40ms/step
0.9107406625870437
75/75 [==============================] - 1s 12ms/step
              precision    recall  f1-score   support

  background       1.00      0.86      0.93      3187
      banana       0.95      0.88      0.92      3320
        wine       0.81      0.99      0.89      2971

    accuracy                           0.91      9478
   macro avg       0.92      0.91      0.91      9478
weighted avg       0.92      0.91      0.91      9478

